# Python Performance Techniques
This notebook explores some techniques for improving the speed of Python programs.
Note the times below each cell, and compare them between implementations.
If you run the same cells on your computer, you'll get different times, but the *ratios* between different implementations should stay about the same. 

## Prefer built-in functions

`'x' in string` (the first cell) uses a built-in function for testing whether a character (or substring) is in a string. The following implementations of `find_char` are all slower.

In [8]:
%%time
for _ in xrange(1000):
    assert 'x' in 'abcdefghijklmnopqrstuvwxyz'

CPU times: user 142 µs, sys: 119 µs, total: 261 µs
Wall time: 181 µs


In [10]:
%%time
def find_char(c, string):
    for c1 in string:
        if c == c1:
            return True
    return False

for _ in xrange(1000):
    assert find_char('x', 'abcdefghijklmnopqrstuvwxyz')

CPU times: user 2.26 ms, sys: 633 µs, total: 2.9 ms
Wall time: 2.56 ms


In [12]:
%%time
def find_char(c, string):
    for i in range(len(string)):
        if string[i] == c:
            return True
    return False

for _ in xrange(1000):
    assert find_char('x', 'abcdefghijklmnopqrstuvwxyz')

CPU times: user 2.61 ms, sys: 848 µs, total: 3.46 ms
Wall time: 2.88 ms


In [14]:
%%time
def find_char(c, string):
    i = 0
    while i < len(string):
        if string[i] == c:
            return True
        i += 1
    return False

for _ in xrange(1000):
    assert find_char('x', 'abcdefghijklmnopqrstuvwxyz')

CPU times: user 4.16 ms, sys: 1.14 ms, total: 5.3 ms
Wall time: 4.4 ms


## Use the right data structure
`set` (and `dict`) are slower than `list` to *create* or *add items to*, but they're faster to test for the presence of values. Which one is faster *overall* depends on how many times you're going to be add items to them, versus test whether they contain values.

The examples below compare a thousand tests for whether a string is in a `list`, to whether it's in a `set`. The `set` case is about 30 times faster. 

In [9]:
import random
import string

def generate_some_random_strings(count=1000):
    """Create a list of random strings. For use in performance testing, below.
    
    Examples:
    >>> random.seed(1)
    >>> generate_some_random_strings(2)
    ['GrmMXVdjEB', 'sZdBLlT']
    >>> generate_some_random_strings(3)
    ['SRaedV', 'yoZKfSaObC', 'wRshXuKgDH']
    """
    strings = []
    letters = string.letters
    for _ in xrange(count):
        s = ''.join(random.sample(letters, len(letters))[:random.randint(5, 10)])
        strings.append(s)
    return strings

import doctest
doctest.run_docstring_examples(generate_some_random_strings, globals())

In [12]:
%%time

random_string_array = generate_some_random_strings()
for _ in xrange(1000):
    s = random.choice(random_string_array)
    assert s in random_string_array

CPU times: user 33.4 ms, sys: 8.32 ms, total: 41.7 ms
Wall time: 36 ms


In the cell above, the `random_string_array` is tested against a number of times.
In this case, it would be faster to use a `set`, to make all the tests faster.

In [11]:
%%time
import random

random_string_array = generate_some_random_strings(2)
random_string_set = set(random_string_array)
for _ in xrange(1000):
    s = random.choice(random_string_array)
    assert s in random_string_set

CPU times: user 870 µs, sys: 614 µs, total: 1.48 ms
Wall time: 1.04 ms


## Cache computed values
One way to make code faster is to not run it so often. A common technique is to store (cache) the result of a computation the first time, and use the cached value from then on.

### Re-use computed values: Hoisting
A common case for cached computation is when the computation is performed inside a loop, but produces the same value each time.

`case_insensitive_membership_test`, below, tests whether a candidate string is a member of a list of strings, ignoring case. (So `"Monday"` and `"monday"` are the same, for purposes of this test.)

In [ ]:
import random
import string

def generate_some_random_strings():
    strings = []
    letters = string.letters
    for _ in xrange(1000):
        s = ''.join(random.sample(letters, len(letters))[:random.randint(5, 10)])
        strings.append(s)
    return strings

In [59]:
%%time
def case_insensitive_membership_test(candidate_string, strings):
    """
    Examples:
    >>> days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    >>> case_insensitive_membership_test('monday', days_of_week)
    True
    >>> case_insensitive_membership_test('TUESDAY', days_of_week)
    True
    >>> case_insensitive_membership_test('wEdNeSdAy', days_of_week)
    True
    >>> case_insensitive_membership_test('February', days_of_week)
    False
    """
    for s in strings:
        if candidate_string.lower() == s.lower():
            return True
    return False

import doctest
#doctest.run_docstring_examples(case_insensitive_membership_test, globals())

strings = generate_some_random_strings() + ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
for _ in xrange(1000):
    assert case_insensitive_membership_test('monday', strings)

CPU times: user 446 ms, sys: 7.14 ms, total: 453 ms
Wall time: 452 ms


The implementation in the cell above computes `candidate_string.lower()` each time through the loop, even though its value stays the same each time.

The implementation in the cell below “[hoists](https://en.wikipedia.org/wiki/Loop-invariant_code_motion)” the computation of `candidate_string.lower()` outside the loop, improves the performance. This implementation is 30% faster.

In [77]:
%%time
def case_insensitive_membership_test(candidate_string, strings):
    """
    Examples:
    >>> days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    >>> case_insensitive_membership_test('monday', days_of_week)
    True
    >>> case_insensitive_membership_test('TUESDAY', days_of_week)
    True
    >>> case_insensitive_membership_test('wEdNeSdAy', days_of_week)
    True
    >>> case_insensitive_membership_test('February', days_of_week)
    False
    """
    candidate_string = candidate_string.lower()
    for s in strings:
        if candidate_string == s.lower():
            return True
    return False

import doctest
#doctest.run_docstring_examples(case_insensitive_membership_test, globals())

strings = generate_some_random_strings() + ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
for _ in xrange(1000):
    assert case_insensitive_membership_test('monday', strings)

CPU times: user 312 ms, sys: 5.83 ms, total: 318 ms
Wall time: 315 ms


We'll return to this function later.

### Re-use computed values: Memoization

`fib`, below, is a straightforward implementation of the [Fibonacci function](https://en.wikipedia.org/wiki/Fibonacci_number). 

In [89]:
%%time
def fib(n):
    """
    Examples:
    >>> fib(0)
    0
    >>> fib(2)
    1
    >>> [fib(i) for i in xrange(10)]
    [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
    """
    if n <= 1:
        global c
        if n == 0: c += 1
        return n
    else:
        return fib(n - 2) + fib(n - 1)

import doctest
#doctest.run_docstring_examples(fib, globals())

fib(30)

CPU times: user 424 ms, sys: 3.75 ms, total: 428 ms
Wall time: 428 ms


Computing `fib(n)` takes time proportional to $\phi^n$, where $\phi$ is the Golden Ration ${1 + \sqrt(5)} \over 2$. This is because a call to `fib(n)` results in one call to `fib(n-1)`,  two calls to `fib(n-2)`, three calls to `fib(n-3)`, *five* calls to `fib(n-4)`, and so on. In other words, it's *really slow*.

Consider a call to `fib(30)`. `fib(28)` is the same both times we call it; `fib(27)` is the same all three times, and so on. Instead of computing each `fib` each time it's called, we can save ("cache") the result, and re-use that. This improves the performance of `fib(30)` by an order of magnitude; the improvement is larger for larger arguments to `fib`.

In [91]:
%%time

fib_cache = {}
# This is a dictionary from `n` to `fib(n)`. It's a global variable, so give it a name
# that is hopefully unique.

# the original fib function, except it calls itself via `cached_fib` instead of directly
def fib(n):
    """
    Examples:
    >>> fib(0)
    0
    >>> fib(2)
    1
    >>> [fib(i) for i in xrange(10)]
    [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
    """
    if n <= 1:
        return n
    else:
        return cached_fib(n - 2) + cached_fib(n - 1)

def cached_fib(n):
    x = fib_cache.get(n)
    if x is None:
        x = fib(n)
        fib_cache[n] = x
    return x

import doctest
#doctest.run_docstring_examples(fib, globals())

fib(30)

CPU times: user 39 µs, sys: 19 µs, total: 58 µs
Wall time: 48.2 µs


This particular variety of caching, where the cache is a dictionary of input values to output values and is used to pre-empt a call to a function that computes those values, is called [memoization](https://en.wikipedia.org/wiki/Memoization).

## Reduce object churn

`range` creates a list. `xrange` creates an object that supplies numbers to a `for` loop, one by one, but never actually makes a place to put all the numbers at once.

Where we don't care about the list itself – just about getting a sequence of values from it – `xrange` can be faster.

Compare the timings below, that iterate over a million elements.

In [3]:
%%time

accum = 0
for i in range(1000 * 1000):
    accum += i ** 2
print accum

333332833333500000
CPU times: user 172 ms, sys: 2.79 ms, total: 175 ms
Wall time: 174 ms


In [4]:
%%time

accum = 0
for i in xrange(1000 * 1000):
    accum += i ** 2
print accum

333332833333500000
CPU times: user 144 ms, sys: 2.76 ms, total: 146 ms
Wall time: 145 ms


In [5]:
%%time
sum(i**2 for i in range(1000 * 1000))

CPU times: user 119 ms, sys: 1.28 ms, total: 120 ms
Wall time: 120 ms


333332833333500000

In [6]:
%%time
sum(i**2 for i in xrange(1000 * 1000))

CPU times: user 96.2 ms, sys: 2.31 ms, total: 98.5 ms
Wall time: 97.4 ms


333332833333500000

## Pre-processing

Now return to `case_insensitive_membership_test`, repeated below.

In [92]:
def case_insensitive_membership_test(candidate_string, strings):
    candidate_string = candidate_string.lower()
    for s in strings:
        if candidate_string == s.lower():
            return True
    return False

Previously we moved the call to `candidate_string.lower()` outside the loop. We can go further, moving the calls to `s.lower()` outside the loop too (and outside the entire function).

We do this by moving these calls into another function, `make_case_insensitive_list`, that's called only once. `make_case_insensitive_list` returns a value that captures the work that's independent of the specific candidate string. (In this case, the value is a list of lowercase strings, that `s.lower()` no longer needs to be applied to.) `search_case_insensitive_list` re-uses this value each time we need to look for a *new* candidate string in the *same* list.

In [60]:
%%time
def make_case_insensitive_list(strings):
    """Construct a data structure that can be searched quickly for a string, ignoring case."""
    return [s.lower() for s in strings]

def search_case_insensitive_list(candidate_string, lowercase_strings):
    """
    Examples:
    >>> days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    >>> case_insensitive_list = make_case_insensitive_list(days_of_week)
    >>> search_case_insensitive_list('monday', case_insensitive_list)
    True
    >>> search_case_insensitive_list('TUESDAY', case_insensitive_list)
    True
    >>> search_case_insensitive_list('wEdNeSdAy', case_insensitive_list)
    True
    >>> case_insensitive_membership_test('February', days_of_week)
    False
    """
    string = string.lower()
    for s in lowercase_strings:
        if candidate_string == s:
            return True
    return False

import doctest
#doctest.run_docstring_examples(search_case_insensitive_list, globals())

strings = generate_some_random_strings() + ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
case_insensitive_list = make_case_insensitive_list(strings)
for _ in xrange(1000):
    assert search_case_insensitive_list('monday', case_insensitive_list)

CPU times: user 73.4 ms, sys: 3.64 ms, total: 77 ms
Wall time: 74.9 ms


This transformation *helps* this specific pattern, where there's *one* list that's searched for *multiple* candidate strings. It would *hurt* in a different usage pattern, where the list to be searched is different each time. (Why?)

Note that this transformation simplifies the body of the `for` loop, to where we can apply the "use a built-in function" strategy:

In [62]:
%%time
def make_case_insensitive_list(strings):
    """Construct a data structure that can be searched quickly for a string, ignoring case."""
    return [s.lower() for s in strings]

def search_case_insensitive_list(candidate_string, lowercase_strings):
    """
    Examples:
    >>> days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    >>> case_insensitive_list = make_case_insensitive_list(days_of_week)
    >>> search_case_insensitive_list('monday', case_insensitive_list)
    True
    >>> search_case_insensitive_list('TUESDAY', case_insensitive_list)
    True
    >>> search_case_insensitive_list('wEdNeSdAy', case_insensitive_list)
    True
    >>> case_insensitive_membership_test('February', days_of_week)
    False
    """
    return candidate_string.lower() in lowercase_strings

import doctest
#doctest.run_docstring_examples(search_case_insensitive_list, globals())

strings = generate_some_random_strings() + ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
case_insensitive_list = make_case_insensitive_list(strings)
for _ in xrange(1000):
    assert search_case_insensitive_list('monday', case_insensitive_list)

CPU times: user 38.9 ms, sys: 5.73 ms, total: 44.6 ms
Wall time: 40.8 ms


And now we can apply "use the right data structure", for the fastest time yet: 30.8 ms, down from an initial 453 ms.

In [65]:
%%time
def make_case_insensitive_set(strings):
    """Construct a data structure that can be searched quickly for a string, ignoring case."""
    return set([s.lower() for s in strings])

def search_case_insensitive_set(candidate_string, lowercase_strings):
    """
    Examples:
    >>> days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    >>> make_case_insensitive_set = make_case_insensitive_list(days_of_week)
    >>> search_case_insensitive_set('monday', case_insensitive_list)
    True
    >>> search_case_insensitive_set('TUESDAY', case_insensitive_list)
    True
    >>> search_case_insensitive_set('wEdNeSdAy', case_insensitive_list)
    True
    >>> search_case_insensitive_set('February', days_of_week)
    False
    """
    return candidate_string.lower() in lowercase_strings

import doctest
#doctest.run_docstring_examples(search_case_insensitive_list, globals())

strings = generate_some_random_strings() + ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
case_insensitive_list = make_case_insensitive_list(strings)
for _ in xrange(1000):
    assert search_case_insensitive_list('monday', case_insensitive_list)

CPU times: user 27.1 ms, sys: 3.73 ms, total: 30.8 ms
Wall time: 28.6 ms
